# Spike History Analysis (LMC minimal dataset)

This notebook uses the cache builder in `scripts/spikehistory_cache_builder.py` to generate per-recording firing-rate and spike-time caches from the minimal analog dataset (`/Volumes/MannySSD/lmc_project_v2_MINIMAL`).

**Important:** The builder writes to a fresh output base; if that path already exists, it aborts to avoid overwriting. Update `output_base` below if needed.

In [2]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
from scripts.spikehistory_cache_builder import SpikeHistoryCacheBuilder

# Point Python to the repo root (one level up from notebooks/)
repo_root = Path.cwd().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Configure paths
DATA_ROOT = Path('/Volumes/MannySSD/lmc_project_v2_MINIMAL')
OUTPUT_BASE = Path('/Volumes/MannySSD/PSTH_regenerated_cache_lmc_run3')

# Initialize builder (does not run yet)
builder = SpikeHistoryCacheBuilder(data_root=DATA_ROOT, output_base=OUTPUT_BASE)


ModuleNotFoundError: No module named 'scripts.spikehistory_cache_builder'

In [ ]:
# Builder initialized above in Cell 1. Use the builder instance from Cell 1.

In [ ]:
# Inspect inputs (paths and availability)
inputs_df = builder.summarize_inputs()

# Inspect an existing cache (after running builder)
# Uncomment to see shapes and stats for up to 3 recordings
# builder.summarize_output(base=OUTPUT_BASE, max_records=3)


In [ ]:
# Example: load one recording's cached data (after builder.run())
# NOTE: run3 already exists; adjust rec_group/rec_name to another cached recording if needed
rec_group = 'Lmc_opsin'  # group_name used in cache path
rec_name = 'lmc_ch_1_3094_rec1'
rec_dir = OUTPUT_BASE / rec_group / rec_name

if not rec_dir.exists():
    raise FileNotFoundError(f"Cache not found at {rec_dir}. Run builder.run() with a fresh OUTPUT_BASE or set rec_group/rec_name to an existing cache.")

meta = pd.read_parquet(rec_dir / 'meta.parquet')
trials = pd.read_parquet(rec_dir / 'trials.parquet')
firing_npz = np.load(rec_dir / 'firing_tensor.npz', allow_pickle=True)
spikes_npz = np.load(rec_dir / 'spikes_relative.npz', allow_pickle=True)

firing_hz = firing_npz['firing_hz']  # shape (channels, trials, time_bins)
time_ms = firing_npz['time_ms']
spikes_rel = spikes_npz['spikes_rel']  # object array [ch, trial] -> spike times (s) relative to onset

print(meta)
print('firing_hz shape:', firing_hz.shape)
print('trials shape:', trials.shape)


In [ ]:
# Detailed inspection for one cached recording (run after builder.run())
rec_group = 'Lmc_opsin'
rec_name = 'lmc_ch_1_3094_rec1'
rec_dir = OUTPUT_BASE / rec_group / rec_name
if not rec_dir.exists():
    raise FileNotFoundError(f"Cache not found at {rec_dir}. Run builder.run() with a fresh OUTPUT_BASE or adjust rec_group/rec_name.")
summary = builder.inspect_recording_cache(rec_dir, verbose=True)
summary


In [ ]:
# Optional: inspect another cached recording (example duplicated; adjust as needed)
# rec_group = 'Lmc_opsin'
# rec_name = 'lmc_ch_1_3094_rec2'
# rec_dir = OUTPUT_BASE / rec_group / rec_name
# if rec_dir.exists():
#     summary = builder.inspect_recording_cache(rec_dir, verbose=True)
#     summary
# else:
#     print(f"Cache not found at {rec_dir}")


In [ ]:
# Quick PSTH plot for the example recording
import matplotlib.pyplot as plt

mean_psth = firing_hz.mean(axis=1).mean(axis=0)  # avg over channels and trials
plt.figure(figsize=(6,3))
plt.plot(time_ms, mean_psth, color='black')
plt.xlabel('Time (ms)')
plt.ylabel('Firing rate (Hz)')
plt.axvline(0, color='red', linestyle='--')
plt.axvline(500, color='red', linestyle='--')
plt.title(f'PSTH: {rec_name}')
plt.show()
